In [3]:
from copy import deepcopy

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import numpy as np

from kaggle_project2.main import optimizer

ModuleNotFoundError: No module named 'kaggle_project2'

In [ ]:
train_rawdata = datasets.MNIST(root='dataset',
                               train=True,
                               download=True,
                               transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='dataset',
                              train=False,
                              download=True,
                              transform=transforms.ToTensor())
print('number of training data : ', len(train_rawdata))
print('number of test data : ', len(test_dataset))

In [ ]:
train_indices, val_indices, _, _ = train_test_split(
    range(len(train_rawdata)),
    train_rawdata.targets,
    stratify=train_rawdata.targets,
    test_size=0.2
)

In [ ]:
train_dataset = Subset(train_rawdata, train_indices)
validation_dataset = Subset(train_rawdata, val_indices)

In [ ]:
print (len(train_dataset), len(validation_dataset), len(test_dataset))

In [ ]:
BATCH_SIZE = 128

train_batches = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_batches = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_batches = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
x_train, y_train = next(iter(train_batches))
print (x_train.shape, y_train.shape)

In [ ]:
index = 1
x_train[index,:,:,:].shape

In [ ]:
x_train[index,:,:,:].numpy().reshape(28,28).shape

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.figure(figsize=(10,12))

x_train, y_train = next(iter(train_batches))
for index in range(100):
    plt.subplot(10,10,index+1)
    plt.axis('off')
    plt.imshow(x_train[index, :, :, :].numpy().reshape(28,28), cmap='gray')
    plt.title('Class: '+str(y_train[index].item()))

In [ ]:
x_train, y_train = next(iter(train_batches))
print (x_train.shape, y_train.shape)
print (x_train.size(0))
print (x_train.view(x_train.size(0), -1).shape)

In [2]:
class FunModel(nn.Module):
    def __init__(self, i, o):
        self.go = nn.Sequential(
            nn.Linear(i, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128, o),
            nn.LogSoftmax(dim=-1)
        )

    def forward(self,x):
        return self.go(x)

batch_size = 128
input_dim = 28*28
output_dim = 10
model = FunModel(input_dim, output_dim)

loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())

NameError: name 'nn' is not defined

In [ ]:
def train_model(model, early_stop, n_epochs, progress_interval):

    train_losses, valid_losses, lowest_loss = [], [], np.inf

    for epoch in range(n_epochs):

        train_loss , valid_loss = 0,0

        model.train()
        for x, y in train_batches:

            y_pred = model(x.view(x.size(0),-1))
            loss = loss_func(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()

        train_loss = train_loss/len(train_batches)
        train_losses.append(train_loss)

        model.eval()
        with torch.no_grad():
            for x, y in val_batches:
                y_pred = model(x.view(x.size(0),-1))
                loss = loss_func(y_pred, y)
                valid_loss +=loss.item()

            valid_loss = valid_loss/len(val_batches)
            valid_losses.append(valid_loss)

            if valid_losses[-1] < lowest_loss:
                lowest_loss = valid_losses[-1]
                lowest_epoch = epoch
                best_model = deepcopy(model.state_dict())

            else:
                if(early_stop>0) and lowest_epoch + early_stop <epoch:
                    print('Early Stopped', epoch, 'epochs')
                    break